# 3 - Mel-Frequency Cepstral Coefficients
## Mel-Frequency Cep-WHAA??
Uhh yeah, pretty much what the title says lol.

![](http://p.fod4.com/p/media/5c597eb60b/uF9MjJo3QIaijySXC4iL_Confused%20Christian%20Bale.gif)

Okay, let's back up about 3 days. I've been doing a bit of light research on how to meaningfully extract features out of a song. In fact, this can probably be generalized to sounds in general because I'm by no means an expert on any of this and I'm just scratching the surface of signal processing for audio, period.

Last post, I hand-picked a few songs of different genres to look at. I looked at 3 plots, the spectrogram, the chromagram, and the tempogram. The spectrogram gives us a sense of treble and bass, the chromagram gives us a sense of the key of the song, and the tempogram gives us a sense of... well, the tempo.

I saw how, for songs with more of a synthesized sound (techno, hip hop), the chromagram had a lack of focus for tones and keys. Synthesized sounds also unnaturally organized the spectrogram into treble and bass regions due to the ability to perform digital filtering. The tempogram gave us information about either how consistent tempo was in the song or how cleanly a tempo-setting instrument could be deciphered. This is all fine and dandy, but what exactly will the features of our model be?

In the spectrogram and chromagram world, what exactly are we going to map as features? A few thoughts that came to my head were:
- _**Frequency Distribution**_: We saw techno and hip hop had really hard hitting bass, so maybe something really elementary we can do is measure how heavy the bass is in any given song. Sames goes for trebles and mids. How exactly do we do this? Do we bucket together certain frequencies? How do we decide what these buckets are?
- _**Keys**_: We saw that techno and hip hop often times lacked the melody and tone to be able to assign a key to songs, but even for other genres, any song can really be sung in any key... does the key really matter? Also, how do we even find the key? Will I have to do more research? Can librosa determine the key for me?
- _**Tempo**_: Even without doing any more deep dives into songs, I already know that House music lives in the 120 - 130 BPM and R&B often times is much slower, so maybe straight BPM can be a feature. One other thing we did notice, though, was that how "clean" or "consistent" a tempo is in a song often dictated genres as well. E.g. Techno music is generally quantized to a beat from a computer's clock / metronome, whereas Jazz was often a bit less consistent because of the mood of the music and analog nature of the instruments involved. How do we measure this, though?

This post will be concentrated on _**frequency distribution**_ because, well, it's one hell of a topic...

## Frequency Distribution - MFCC
How do we measure which frequencies exist in a song? Frequencies can pop up anywhere at any time. We saw in Dancing! (under the Dance / Techno category in the last post) that the intro had no bass whatsoever, but immediately after the intro, we had very heavy bass. For Techno, we would hope to see heavy bass in whatever method we use to track the footprint of frequencies. Timing is another thing that comes to my mind, though. What if we took the same sequences of verses, buildups, and drops from Dancing! and just switched them around. They would be a different song, but should still have that techno feel. At the end of the day, we simply want to categorized the genres and not the song itself.

After 3 of light research, let's fast forward back to the Mel-Frequency Cepstrum Coefficient (MFCC). This seems to be a very popular method of characterizing frequencies in _**speech**_ recognition systems, and it also seems to be gaining traction in the music world. It's quite the journey to understand exactly what an MFCC represents, but I will try my best to capture everything (that I understand).

###